In [1]:
from agent_nodes import g2i

import rclpy
from rclpy.node import Node
from context_based_gesture_operation.srv import G2I
from context_based_gesture_operation.msg import Intent
from context_based_gesture_operation.msg import Scene as SceneRos
from context_based_gesture_operation.msg import Gestures as GesturesRos
import numpy as np
from srcmodules.Scenes import Scene
from srcmodules.Gestures import Gestures
from srcmodules.Actions import Actions
from srcmodules.SceneFieldFeatures import SceneFieldFeatures

#  Behaviour tree example

- `roscore` need to be running in the backend
- `rosrun rqt_py_trees rqt_py_trees` to visualize the tree

### Init Gestures to Intent Service

In [2]:
A = ['move_up', 'move_left', 'move_down', 'move_right',
         'put', 'put_on', 'pour', 'pick_up', 'place', 'open', 'close']
G = ['swipe_up', 'swipe_left', 'swipe_down', 'swipe_right',
          'five', 'grab', 'thumbsup', 'rotate', 'point', 'two', 'three']

In [3]:
rclpy.init()

g2i.init()
g2i.rosnode = g2i.G2IRosNode(G, A)

### Init scene & gesture vector publishers

In [4]:
pub_scene = g2i.rosnode.create_publisher(SceneRos, "/tree/scene_in", 5)
pub_gestures = g2i.rosnode.create_publisher(GesturesRos, "/tree/gestures_in", 5)

### Save the robotic actions

In [5]:
robotic_sequence = []
def save_the_robotic_sequence(msg):
    global robotic_sequence
    robotic_sequence.append(msg)
g2i.rosnode.create_subscription(Intent, '/execute_intent', save_the_robotic_sequence, 5)

### Create B Tree

In [6]:
from srcmodules import BTreeLib
import py_trees_ros, functools

In [7]:
root = BTreeLib.create_tree()
behaviour_tree = py_trees_ros.trees.BehaviourTree(root)
#rclpy.on_shutdown(functools.partial(BTreeLib.shutdown, behaviour_tree))

#if not behaviour_tree.setup(timeout=15):
#    console.logerror("failed to setup the tree, aborting.")
#    sys.exit(1)

In [8]:
behaviour_tree.tick()

update scene: Status.RUNNING


# Ex. 1 
- put to drawer, when drawer is closed

1. Generate Scene and Gesture vector
2. Publish Scene and gesture vector info and generate the robotic action sequence 

In [12]:
g = GesturesRos()
g.probabilities.data = list(np.array(np.zeros(11), dtype=float))
g.probabilities.data[5] = 1.0

s = Scene(init='object,drawer,cup', random=False)
Actions.do(s,('pick_up','cup'), ignore_location=True)
sr = s.to_ros(SceneRos())
sr.focus_point = np.array(s.drawer.position, dtype=float)

In [13]:
pub_scene.publish(sr)
pub_gestures.publish(g)

In [14]:
behaviour_tree.tick()

update scene: Status.RUNNING


In [15]:
robotic_sequence

[]

# Ex. 2
- move left

In [18]:
robotic_sequence = []

g = GesturesRos()
g.probabilities.data = list(np.array(np.zeros(11), dtype=float))
g.probabilities.data[1] = 1.0

s = Scene(init='object,drawer,cup', random=False)
Actions.do(s,('pick_up','cup'), ignore_location=True)
sr = s.to_ros(SceneRos())
sr.focus_point = np.array(s.object.position, dtype=float)

pub_scene.publish(sr)
pub_gestures.publish(g)

behaviour_tree.tick()
robotic_sequence

update scene: Status.RUNNING


[]

# Ex. 3
- put to drawer, drawer is opened

In [20]:
robotic_sequence = []

g = GesturesRos()
g.probabilities.data = list(np.array(np.zeros(11), dtype=float))
g.probabilities.data[5] = 1.0

s = Scene(init='object,drawer,cup', random=False)
s.drawer.open()
Actions.do(s,('pick_up','cup'), ignore_location=True)
sr = s.to_ros(SceneRos())
sr.focus_point = np.array(s.drawer.position, dtype=float)

pub_scene.publish(sr)
pub_gestures.publish(g)

behaviour_tree.tick()
robotic_sequence

update scene: Status.RUNNING


[]

1667902431.976800 [0]        tev: ddsi_udp_conn_write to udp/10.37.1.84:55295 failed with retcode -1
1667902432.075885 [0]        tev: ddsi_udp_conn_write to udp/239.255.0.1:7400 failed with retcode -1
1667902432.075899 [0]        tev: ddsi_udp_conn_write to udp/10.37.1.84:55295 failed with retcode -1
1667902432.076854 [0]        tev: ddsi_udp_conn_write to udp/10.37.1.84:55295 failed with retcode -1
1667902432.176939 [0]        tev: ddsi_udp_conn_write to udp/10.37.1.84:55295 failed with retcode -1
1667902432.277007 [0]        tev: ddsi_udp_conn_write to udp/10.37.1.84:55295 failed with retcode -1
1667902432.382129 [0]        tev: ddsi_udp_conn_write to udp/10.37.1.84:55295 failed with retcode -1
1667902432.482232 [0]        tev: ddsi_udp_conn_write to udp/10.37.1.84:55295 failed with retcode -1
1667902432.582354 [0]        tev: ddsi_udp_conn_write to udp/10.37.1.84:55295 failed with retcode -1
1667902432.782504 [0]        tev: ddsi_udp_conn_write to udp/10.37.1.84:55295 failed with r

# Ex. 4
- put to drawer, not holding anything, target drawer opened
    
- Unfeasible, no object held in gripper -> action discarded before when checking action validity 